<a href="https://colab.research.google.com/github/heispv/projects/blob/master/BSc/BSc_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing modules

In [11]:
import numpy as np
import math as m
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets import interact

# Using an interactive GUI

In [12]:
def Reactor(H2_init=0.4325, H2O_init=0.0002, CO_init=0.1716, CO2_init=0.0409,
            CH3OH_init=0.003, DME_init=0.0018, CH4_init=0.044, N2_init=0.316,
            P_init=50, T_init=493, Tc=513.15, Lr=5.8, ros=1200, dt=0.038, dout=0.0422):
  
  Tc = Tc
  Lr = Lr
  ros = ros
  dt = dt
  dout = dout


  def diff_eq(Y, L):

    XH2 = Y[0]
    XH2O = Y[1]
    XCO = Y[2]
    XCO2 = Y[3]
    XCH3OH= Y[4]
    XDME = Y[5]
    XCH4 = Y[6]
    XN2 = Y[7]
    PT = Y[8]
    T = Y[9]

    # Constants
    Ks = 4.18           #conductivity coefficent pellet(kw/m.S)-->(w/m.s)(table3.3)
    NP = 4177           #number of pipes(table4)
    ebs = 0.455         #porosity(table4)
    Dp = 0.0055         #diameter of catalyst particle(m)(email)
    Kw = 48             #conductivity coefficent gas(kw/m.S)-->(w/m.s)(table3.3)
    Prc = 0.843         #prantle(table3.3)
    Rec = 1923          #critical reynolds(table3.3)
    R = 8.314           #universal gas constant(j/mol.k)=(kg.m2/s2.mol.k)=(m3.pa/mol.k)
        
    # Stocumetric coeficient
    nu = np.mat('-2 -3 0; 0 1 1; -1 0 0; 0 -1 0; 1 1 -2; 0 0 1; 0 0 0; 0 0 0')
                #H2      #H2O    #CO    #CO2    #CH3OH  #DME   #CH4   #N2
                
    # Molecular weight(kg/kmol)
    MW = np.mat('2.016; 18.016; 28.01; 44.01; 32.042; 46.069; 16.042; 28.01')
                #H2    #H2O    #CO    #CO2   #CH3OH  #DME    #CH4    #N2


    # Algebric
    PCO = PT * Y[2]
    PH2O = PT * Y[1]
    PCH3OH = PT * Y[4]
    PH2 = PT * Y[0]
    PCO2 = PT * Y[3]
    PN2 = PT * Y[7]
    PDME = PT * Y[5]
    PCH4 = PT * Y[6]

    # cp(j/mol.k)
    cpco = 30.87 - 1.285e-2*T + 2.789e-5*T**2 - 1.272e-8*T**3
    cpco2 = 19.8 + 7.344e-2*T - 5.602e-5*T**2 + 1.715e-8*T**3
    cpch3och3 = 10.702 + 1.791e-1*T - 5.234e-5*T**2 - 1.918e-9*T**3
    cpch3oh = 21.15 + 7.092e-2*T + 2.587e-5*T**2 - 2.852e-8*T**3
    cph2o = 32.24 + 1.924e-3*T + 1.055e-5*T**2 - 3.596e-9*T**3
    cph2 = 27.14+9.274e-3*T - 1.381e-5*T**2 + 7.645e-9*T**3
    cpn2 = R * (3.28 + 0.593e-3*T + 0.04e5/T**2)
    cpch4 = 19.25 + 5.213e-2*T + 1.197e-5*T**2 - 1.132e-8*T**3

    
    CPg = cph2*XH2 + cph2o*XH2O + cpco*XCO + cpco2*XCO2 + cpch3oh*XCH3OH + \
                cpch3och3*XDME + cpch4*XCH4 + cpn2*XN2 # %J/mol.k

    #Di=.046;         # inner diameter(m)
    #Do=.05;          # outer diameter(m)
    #Ac=NP*(pi*Di**2/4);  # cross section
    
    b=1

    #(j)
    delH1 = (-90850 +((-451/10*T+3369207/20/m.sinh(3833/2/T)*m.cosh(3833/2/T)
        -9621591/200*m.sinh(8967/10/T)/m.cosh(8967/10/T)-27056327/1000/m.sinh(30851/10/T)*m.cosh(30851/10/T)
        +3253293/250*m.sinh(7691/5/T)/m.cosh(7691/5/T)-1178748/25/m.sinh(2466/T)*m.cosh(2466/T)
        +533544/125*m.sinh(2838/5/T)/m.cosh(2838/5/T))-(5.0036e4)))/b
    delH2 = (-50100 +((-2380849/60050*T+83922354/1201/m.sinh(5221/2/T)*m.cosh(5221/2/T)
        -12484920/1201*m.sinh(1169/T)/m.cosh(1169/T)+3369207/20/m.sinh(3833/2/T)*m.cosh(3833/2/T)
        -9621591/200*m.sinh(8967/10/T)/m.cosh(8967/10/T)-1232721/25/m.sinh(1428/T)*m.cosh(1428/T)
        +388212/25*m.sinh(2941/5/T)/m.cosh(2941/5/T)-1768122/25/m.sinh(2466/T)*m.cosh(2466/T)+
        800316/125*m.sinh(2838/5/T)/m.cosh(2838/5/T))-(6.9286e4)))/b
    delH3 = (-23400 +((29998441/4811206*T+83922354/1201/m.sinh(5221/2/T)*m.cosh(5221/2/T)
        -12484920/1201*m.sinh(1169/T)/m.cosh(1169/T)+462420560/2003/m.sinh(8017/5/T)*m.cosh(8017/5/T)
        -112393476/2003*m.sinh(3627/5/T)/m.cosh(3627/5/T)-3369207/10/m.sinh(3833/2/T)*m.cosh(3833/2/T)
        +9621591/100*m.sinh(8967/10/T)/m.cosh(8967/10/T))-(-4.2089e3)))/b

    # deltaH
    CT = (PT*(10**5)) / (R*T)           ##(mol/m**3)eq 3.19
    Rr = dt / 2                        ##(m)
    MWg = MW.transpose()*np.mat(f'{XH2};{XH2O};{XCO};{XCO2};{XCH3OH};{XDME};{XCH4};{XN2}'); #(kg/kmol) dimension
    rog = PT*10**2 * MWg / (R*T)          ## density of fluid phase (kg/m**3)
    M = (2.04*10**(5)) * (np.mat('0.4325 0.0002 0.1716 0.0409 0.003 0.0018 0.044 0.316')*MW) \
                    / 1000/(273*8.314*10**(-5))/3600 ##mass stream(kg/s)
    A = m.pi * Rr**2 * NP           ## cross section(m**2)
    Mt = M / A                 ## mass flux(kg/m**2.s)
    #Mcat=pi*dt**2*Lr*(1-ebs)*rocat*NP/4;
    ug = Mt / rog        ## gas velocity(m/s) :eq 3.21

    kH2 = (0.228117e-02*T**0.745200/(1+12.0000/T+0.000000e+00/T**2))*4.186/3600*1000     #[W/(m.K)]
    kCO2 = (3.17283*T**-0.383800/(1+964.000/T+0.186000e+07/T**2))*4.186/3600*1000        #[W/(m.K)]
    kN2 =(0.284979e-03*T**0.772200/(1+16.3230/T+373.720/T**2))*4.186/3600*1000           #[W/(m.K)]
    kCH4 = (5438.69*T**0.430410/(1+0.770400E+09/T+-0.387250E+11/T**2))*4.186/3600*1000   #[W/(m.K)]
    kCH3OH = (-6.67498*T**1.02790/(1+-0.743600e+08/T+0.677000e+10/T**2))*4.186/3600*1000 #[W/(m.K)]
    kH2O = (0.185778e-02*T**0.768390/(1+3940.50/T+-445340/T**2))*4.186/3600*1000         #[W/(m.K)]
    kCO = (0.514893e-03*T**0.686300/(1+57.1300/T+501.920/T**2))*4.186/3600*1000          #[W/(m.K)]
    kDME = (0.515692e-01*T**0.266700/(1+1018.60/T+0.109880e+07/T**2))*4.186/3600*1000    #[W/(m.K)]

    kgg = np.transpose(np.mat(f'{kH2},{kH2O},{kCO},{kCO2},{kCH3OH},{kDME},{kCH4},{kN2}'))
    Kg = Y[0:8]*kgg

    Ksg = Ks / Kg
    Rp = Dp / 2 #(m)

    muH2 = (2.14524642E-6+2.54245E-8*T**1-1.0235587E-11*T**2+2.80895021E-15*T**3)                   # pa.s
    muCO2 = (-1.48502597E-6+6.46786388E-8*T**1-3.66170537E-11*T**2+1.24501217E-14*T**3)             # pa.s
    muN2 = (1.77230303E-6+6.27427545E-8*T**1-3.47278555E-11*T**2+1.01243201E-14*T**3)               # pa.s
    muCH4 = (-6.811519E-7+4.923719E-8*T**1-3.850654E-11*T**2+2.417259E-14*T**3-6.598298E-18*T**4)   # pa.s
    muCH3OH = (-2.648876E-7+3.09544E-8*T**1+1.295145E-11*T**2-1.944318E-14*T**3+7.149205E-18*T**4) # pa.s
    muH2O = (3.798134E-6+2.778857E-9*T**1+7.621016E-11*T**2-6.580738E-14*T**3+2.07707E-17*T**4)    # pa.s  
    muCO = (0.11127e-2*T**0.5338)/(1+94.7/T)*0.001                                                  # pa.s 
    muDME = (0.268e-2*T**0.3975)/(1+534/T)*0.001                                                    # pa.s

    muegg = np.mat(f'{muH2},{muH2O},{muCO},{muCO2},{muCH3OH},{muDME},{muCH4},{muN2}')
    mueg = np.transpose(Y[0:8])*np.transpose(muegg) #(kg/m.s)=(pa.s)
    
    Pr = (mueg*CPg/Kg)*1000/(np.transpose(MW)*np.mat(f'{XH2};{XH2O};{XCO};{XCO2};{XCH3OH};{XDME};{XCH4};{XN2}'))
    mueg = float(mueg)
    Re = rog * ug * Lr/mueg
    Kstat = Kg * ((0.5+0.493*(Ksg-1)/(22+Ksg))*(Ksg/(Ksg*ebs+1-ebs))+1-(0.5+0.493*(Ksg-1)/(22+Ksg))*(ebs+((1-ebs)*Ksg)))
    Kdyn = Kg * Re * Pr / ((7+135.8 * Rp/Rr)**2) #(w/m.k)
    Krg = (Kstat + Kdyn)  #effective radial conductivity gas phase(kw/m.k)-->(w/m.k)
    hig = 1.23 * Re * 0.53 * Kg / (2*Rp)  #gas phase heat transfer coefficient,inside tube wall(kw/m**2.k)-->(w/m2.k)
    Rry = dout/2  #reactor tube radius outside(m)
    Krs = Kstat #effective radial conductivity solid phase(kw/m k)-->(w/m2.k)
    hoc = (0.02*Rec**2*(Prc**(1/3))*(Rry/Rr)**0.53*Kw)/(2*Rry)  #gas phase heat transfer coefficient,outsidetube wall(kw/m**2.k)-->(w/m2.k)
    his = (0.48+0.192*((Rr/Rp-1)**2)*Krs)/Rp  #solid phase heat transfer coefficient,insidetube wall(kw/m**2.k)-->(w/m2.k)
    Uws = np.linalg.inv((1/his)+(Rr/(Rry*hoc))+(Rr*np.log(Rry/Rr)/(2*Kw)))
    Uwg = np.linalg.inv((1/hig)+(Rr/(Rry*hoc))+(Rr*np.log(Rr/Rry)/(2*Kw)))
    UT = np.linalg.inv(1/Uwg+(3*Rr/(8*Krg)))+np.linalg.inv(1/Uws+(3*Rr/(8*Krs)))  #overall heat transfer coefficient(W/m**2 k)

    # Safari

    kf1 = np.exp(22.7+8.975e3/T-7.694*np.log(T)+3.92e-3*T+5.123e-7*T**2-3.114e-10*T**3)
    kf2 = np.exp(17.6+4.213e3/T-5.752*np.log(T)-1.707e-3*T+2.682e-6*T**2-7.232e-10*T**3)
    kf3 = np.exp(-9.39+3.205e3/T+.836*np.log(T)+2.353e-3*T-1.874e-6*T**2+5.160e-10*T**3)

    # shim

    #kf1=np.exp(4213/T-5.752*np.log(T)-1.707e-3*T+2.682e-6*(T**2)-7.232e-10*(T**3)+17.6)
    #kf2=np.exp(4213/T-5.752*np.log(T)-1.707e-3*T+2.682e-6*(T**2)-7.232e-10*(T**3)+17.6)
    #kf3=np.exp(4019/T+3.707*np.log(T)-2.783e-3*T+3.8e-7*(T**2)-6.561e4/(T**3)-26.64)

    betta1 = (PT*(Y[4]))/(kf1*PT**3*(Y[2])*(Y[0])**2)
    betta2 = (PT**2*(Y[4])*(Y[1]))/(kf2*PT**4*(Y[3])*(Y[0])**3)
    betta3 = (PT**2*(Y[5])*(Y[1]))/(kf3*PT**2*(Y[4])**2)

    # mevawala-nie
    k11=7.38e3*np.exp(-66557/(R*T))
    k12=4.459e3*np.exp(-74105/(R*T))
    k13=4.062e3*np.exp(-41973/(R*T))
    kco=5.134e-6*np.exp(38773/(R*T))    
    kco2=1.758e-6*np.exp(52395/(R*T))
    kh2=.6716*np.exp(-6476/(R*T))
    kch3oh=3.48e-6*np.exp(54689/(R*T)) 


    a = 1
    r1 = 0.35*((k11*PT**3*(Y[2])*(Y[0])**2*(1-betta1))/(1+kco*PT*(Y[2])+kco2*PT*(Y[3])+kh2*PT*(Y[0]))**3)*(1000/3600)*a   #r1=rco(mol/kgcat.s)
    r2 = 0.35*((k12*PT**4*(Y[3])*(Y[0])**3*(1-betta2))/(1+kco*PT*(Y[2])+kco2*PT*(Y[3])+kh2*PT*(Y[0]))**4)*(1000/3600)*a   #r2=co2
    r3 = 0.35*((k13*PT*(Y[4])*(1-betta3))/(1+np.sqrt(kch3oh*PT*(Y[4])))**2)*(1000/3600)*a   #r3=rDME
    r = np.array((r1, r2, r3))

    # function
    # X = mole fraction
    r = np.expand_dims(r, axis=0).transpose()
    dXH2 = ((Lr*ros/float(ebs*CT*ug))*(nu[0]*r-XH2*np.ones((1,8))*(nu*r)))
    dXH2O = ((Lr*ros/(ebs*CT*ug))*(nu[1]*r-XH2O*np.ones((1,8))*(nu*r)))
    dXCO = ((Lr*ros/(ebs*CT*ug))*(nu[2]*r-XCO*np.ones((1,8))*(nu*r)))
    dXCO2 = ((Lr*ros/(ebs*CT*ug))*(nu[3]*r-XCO2*np.ones((1,8))*(nu*r)))
    dXCH3OH = ((Lr*ros/(ebs*CT*ug))*(nu[4]*r-XCH3OH*np.ones((1,8))*(nu*r)))
    dXDME = ((Lr*ros/(ebs*CT*ug))*(nu[5]*r-XDME*np.ones((1,8))*(nu*r)))
    dXCH4 = ((Lr*ros/(ebs*CT*ug))*(nu[6]*r-XCH4*np.ones((1,8))*(nu*r)))
    dXN2 = ((Lr*ros/(ebs*CT*ug))*(nu[7]*r-XN2*np.ones((1,8))*(nu*r)))
    dPT = -(10**-5)*Lr*float(1.75+150*((1-ebs)/(Dp*rog*ug*ebs/mueg)))*float(((ug*ebs)**2)*rog/Dp)*((1-ebs)/ebs**3)  #(bar)
    dT = float(A/(CPg)*((np.pi*dt*UT/A)*(((T+Tc)/2)-T)+rog*0.7*(r1*(-delH1)+r2*(-delH2)+r3*(-delH3))))
    

    dy = np.array([float(dXH2), float(dXH2O), float(dXCO), float(dXCO2), float(dXCH3OH), float(dXDME), 
                    float(dXCH4), float(dXN2), float(dPT), float(dT)])



    return dy


  # Intial condition and the steps of gride
  # Y0 = [0.4325,0.0002,0.1716,0.0409,0.003,0.0018,0.044,0.316,50,493]
  Y0 = [H2_init ,H2O_init ,CO_init ,CO2_init ,CH3OH_init ,DME_init ,CH4_init ,N2_init ,P_init ,T_init]
  L  =  np.linspace(0, 1, 10000)

  # using `odeint`
  ans = odeint(diff_eq, Y0, L)

  # Plotting the curves
  C_A = ans[:, 0] 
  C_B = ans[:, 1] 
  C_C = ans[:, 2] 
  C_D = ans[:, 3] 
  C_E = ans[:, 4] 
  C_F = ans[:, 5] 
  C_G = ans[:, 6]
  C_H = ans[:, 7] 
  P = ans[:, 8]
  T = ans[:,9]


  plt.style.use('bmh')

  plt.figure(figsize=(13, 17))

  plt.subplot(2, 1, 1)
  plt.plot(L, C_A,
          L, C_B,
          L, C_C,
          L, C_D,
          L, C_E,
          L, C_F,
          L, C_G,
          L, C_H)
  plt.legend(["H2", "H2O", "CO", "CO2", "CH3OH", "DME", "CH4", "N2"])
  plt.title("Mole fraction change", fontsize=17);
  plt.ylabel("Mole Fraction");
  plt.xlabel("Length");

  plt.subplot(2, 1, 2)
  plt.plot(L, T)
  plt.title("Temperature change", fontsize=17);
  plt.ylabel("Temperature(K)");
  plt.xlabel("Length");



In [13]:
interact(Reactor,
         H2_init=(0.1, 0.8, 0.01),
         H2O_init=(0.001, 0.01, 0.001),
         CO_init=(0.1, 0.4, 0.01),
         CO2_init=(0.01, 0.1, 0.002),
         CH3OH_init=(0.001, 0.01, 0.0002),
         DME_init=(0.001, 0.005, 0.0002),
         CH4_init=(0.01, 0.08, 0.002),
         N2_init=(0.1, 1, 0.02),
         P_init=(25, 75, 2),
         T_init=(300, 600, 5),
         Tc=(400, 650, 5),
         Lr=(1, 10, 0.1),
         ros=(800, 1500, 10),
         dt=(0.01, 0.1, 0.001),
         dout=(0.01, 0.1, 0.001))

interactive(children=(FloatSlider(value=0.4325, description='H2_init', max=0.8, min=0.1, step=0.01), FloatSlid…

<function __main__.Reactor(H2_init=0.4325, H2O_init=0.0002, CO_init=0.1716, CO2_init=0.0409, CH3OH_init=0.003, DME_init=0.0018, CH4_init=0.044, N2_init=0.316, P_init=50, T_init=493, Tc=513.15, Lr=5.8, ros=1200, dt=0.038, dout=0.0422)>